In [1]:
%load_ext autoreload
%autoreload 2
#!pip install sb3-contrib

In [2]:
from sb3_contrib import MaskablePPO
from apad_env import APADEnv
from stable_baselines3.common.callbacks import BaseCallback
import time

In [3]:
class TimerCallback(BaseCallback):
    def __init__(self):
        super().__init__()
        self.start_time = time.time()
    
    def _on_step(self):
        if self.num_timesteps % 1000 == 0:
            elapsed = time.time() - self.start_time
            rate = self.num_timesteps / elapsed
            remaining = (self.locals['total_timesteps'] - self.num_timesteps) / rate
            print(f"Step {self.num_timesteps}, {elapsed:.0f}s elapsed, {remaining:.0f}s remaining")
        return True

In [4]:
env = APADEnv()

In [10]:
model = None
model = MaskablePPO(
    "MlpPolicy",
    env,
    learning_rate=5e-5,
    tensorboard_log="./maskable_ppo_logs_3/",
    verbose=1,
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
total_timesteps = 300000
checkpoint_interval = 50000
for i in range(0, total_timesteps, checkpoint_interval):
    remaining_steps = min(checkpoint_interval, total_timesteps - i)
    if i == 0:
        model.learn(total_timesteps=remaining_steps, reset_num_timesteps=True, callback=TimerCallback())
    else:
        model.learn(total_timesteps=remaining_steps, reset_num_timesteps=False, callback=TimerCallback())
    model.save(f"checkpoint_3_{i}")

Logging to ./maskable_ppo_logs_3/PPO_1


In [12]:
model.save("mppo_model_150k_2025-06-12_1600")